# Detect Trucks Sentinel-2 - Europe
________________                                            

In [2]:
# load creds
%load_ext dotenv
%dotenv

The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


_______________
## 1 | Setup

In [3]:
def install_package(pkg):
    subprocess.check_call([sys.executable, "-m", "pip", "install", pkg])

# general
import os
import datetime
import pandas as pd
import subprocess
import sys
from datetime import date, datetime, timedelta

# installations
install_package("OSMPythonTools")
install_package("geocube")

# OSM API
from OSMPythonTools.overpass import overpassQueryBuilder, Overpass

# xcube
from xcube_sh.cube import open_cube
from xcube_sh.config import CubeConfig
from xcube.core.maskset import MaskSet

# spatial
import xarray as xr
import numpy as np
import geopandas as gpd
import geocube
from rasterio import features
from affine import Affine
from shapely import geometry, coords
from shapely.geometry import Polygon, Point
#from osgeo import gdal #, gdal_array, ogr

# plotting
import matplotlib as plt
import IPython.display
%matplotlib inline

____
## 2 | General Parameters

#### Cube

In [4]:
dataset = "S2L2A"
spatial_res = 0.00009 # approx. 10m
bands = ["B02", "B03", "B04", "B08", "B11", "SCL"]
day_bins = "1D" # for cube
tile_size = [512, 512]
minimum_valid_observations = 30 # percent
grid_spacing = 1. # processing grid box size [degree]

#### Directories

In [5]:
dir_main = os.getcwd()
dir_not_commit = os.path.join(dir_main, "not_commit")
dir_ancil = os.path.join(dir_not_commit, "ancillary_data")
dirs = {"dir":dir_main, "dir_not_commit":dir_not_commit, "ancil":dir_ancil, "processing":os.path.join(dir_main, "processing"),
       "processed":os.path.join(dir_not_commit, "processed"), "ancil_roads":os.path.join(dir_ancil, "roads"), "ancil_gadm":os.path.join(dir_ancil, "gadm")}
for directory in list(dirs.values()):
    if not os.path.exists(directory): os.mkdir(directory)

#### Files

In [6]:
files = {"gadm_efta":os.path.join(dirs["ancil_gadm"], "gadm36_0_EU_EFTA.gpkg"), 
         "gadm_europe":os.path.join(dirs["ancil_gadm"], "gadm36_0_europe.gpkg"), 
         "gadm_europe_union":os.path.join(dirs["ancil_gadm"], "gadm0_europe_union.gpkg"),
         "proc_grid":os.path.join(dirs["processing"], "processing_grid_gadm_%s.geojson" %(grid_spacing))}

#### Temporal

In [96]:
weekdays = ["tuesday", "wednesday", "thursday"] # process mean of these weekdays
target = {"first":datetime(2020, 3, 16), "last":datetime(2020, 6, 10)}
baseline = {"first":datetime(2017, 12, 15), "last":datetime(2020, 3, target["first"].day - 1)}
baseline_years = [2018, 2019] # if processing for a yearly period as in target
n_days_sub = 91 # days per timestamp (sub-period)

#### Area

#### OSM

In [8]:
osm_key = "highway"
osm_values = ["motorway", "trunk", "primary", "secondary"]
roads_buffer = 0.00022 # degree, for motorway, the others lower

______
## 3 | Detection Parameters

In [9]:
thresholds = {"min_rgb":0.04,
              "min_blue":0.1, # treat shadows
              "max_red":0.15,
              "max_green":0.15,
              "max_blue":0.4,
              "max_ndvi":0.7,
              "max_ndwi":0.001,
              "max_ndsi":0.0001,
              "min_b11":0.05,
              "max_b11":0.55,
              "min_green_ratio":0.05,
              "min_red_ratio":0.1}

________________
## 4 | Utils

#### Names

In [10]:
def EPSG_4326(): return "EPSG:4326"
def EPSG_3857(): return "EPSG:3857"
def GEOJSON(): return "GeoJSON"
def GPKG(): return "GPKG"
def GEOJSON_EXT(): return ".geojson"
def GPKG_EXT(): return ".gpkg"
def NC_EXT(): return ".nc"
def BBOX_ID(): return "bbox_id"
def ABS_N_TRUCKS(): return "acquisitions_trucks"
def SUM_TRUCKS(): return "period_sum_trucks"
def MEAN_N_TRUCKS(): return "period_mean_trucks"
def MEAN_N_TRUCKS_VEC(): return "period_mean_trucks_vectorized"
def N_OBS(): return "n_observations"

#### File names

In [11]:
def fname_osm(directory, bbox_id, osm_key, ext = GPKG_EXT()): return os.path.join(directory, str(bbox_id) + "_" + osm_key + ext)
def construct_fname(dirs_ts, dir_ts_key, bbox_id, ext):
    return os.path.join(dirs_ts[dir_ts_key], dir_ts_key + "_" + BBOX_ID() + str(bbox_id) + ext)
def fname_acquisition_trucks(dirs_ts, bbox_id, date, ext = NC_EXT()): 
    return os.path.join(dirs_ts[ABS_N_TRUCKS()], str(date) + "_" + ABS_N_TRUCKS() + "_" + BBOX_ID() + str(bbox_id) + ext)
def fname_sum_trucks(dirs_ts, bbox_id, ext = NC_EXT()): 
    return construct_fname(dirs_ts, SUM_TRUCKS(), bbox_id, ext)
def fname_mean_trucks(dirs_ts, bbox_id, ext = NC_EXT()): 
    return construct_fname(dirs_ts, MEAN_N_TRUCKS(), bbox_id, ext)
def fname_mean_trucks_vec(dirs_ts, bbox_id, ext = GPKG_EXT()): 
    return construct_fname(dirs_ts, MEAN_N_TRUCKS_VEC(), bbox_id, ext)
def fname_mean_trucks_vec_placeholder(dirs_ts, bbox_id, ext = ".txt"): 
    return construct_fname(dirs_ts, MEAN_N_TRUCKS_VEC(), bbox_id, ext)
def fname_sum_obs(dirs_ts, bbox_id, ext = NC_EXT()): 
    return construct_fname(dirs_ts, N_OBS(), bbox_id, ext)

#### Directory structure

In [12]:
def make_dirs_ts(dir_ts):
    dir_ts_overall = os.path.join(dir_ts, "overall")
    if not os.path.exists(dir_ts_overall): os.mkdir(dir_ts_overall)
    dirs_ts = {ABS_N_TRUCKS():os.path.join(dir_ts, ABS_N_TRUCKS()), # acquisition-wise
               SUM_TRUCKS():os.path.join(dir_ts_overall, SUM_TRUCKS()), # aggregation
               MEAN_N_TRUCKS():os.path.join(dir_ts_overall, MEAN_N_TRUCKS()), # aggr
               MEAN_N_TRUCKS_VEC():os.path.join(dir_ts_overall, MEAN_N_TRUCKS_VEC()), # aggr
               N_OBS():os.path.join(dir_ts_overall, N_OBS())} # aggr
    for direc in dirs_ts.values():
        if not os.path.exists(direc): os.mkdir(direc)
    return dirs_ts

#### Processing grid
Create a grid covering Europe for processing chunk-wise.

In [13]:
def make_grid(grid_spacing, files):
    crs = EPSG_4326()
    gadm0_eu_efta = gpd.read_file(files["gadm_efta"])
    xmin,ymin,xmax,ymax = gadm0_eu_efta.total_bounds
    width = xmax - xmin
    height = ymax - ymin
    cols = int((width) / grid_spacing)
    rows = int((height) / grid_spacing)
    box_width = width / cols
    box_height = height / rows
    boxes = []
    for row in range(rows):
        for col in range(cols):
            col_right = col + 1
            row_lower = row + 1
            y_up = ymax-row*box_height
            y_low = ymax-row_lower*box_height
            x_left = xmin+col*box_width
            x_right = xmin+col_right*box_width
            boxes.append(Polygon([(x_left, y_up), 
                                  (x_right, y_up), 
                                  (x_right, y_low), 
                                  (x_left, y_low)]))
    grid = gpd.GeoDataFrame({"geometry":boxes})
    grid.crs = crs
    gadm0_europe = gpd.read_file(files["gadm_europe"])
    gadm0_europe.crs = crs
    file_union = files["gadm_europe_union"]
    if os.path.exists(file_union):
        gadm0_europe_clip_union = gpd.read_file(file_union)
    else:
        gadm0_europe["continent"] = ["EUROPE"] * len(gadm0_europe)
        gadm0_europe_clip = gpd.overlay(gadm0_europe, gpd.GeoDataFrame({"a":[1],"geometry":test}), how = "intersection")
        gadm0_europe_clip_union = gadm0_europe_clip.dissolve(by = "continent")
        gadm0_europe_clip_union.to_file(file_union, driver = GPKG())
    # intersect with gadm for cutting boxes at coast line
    grid_intersect = gpd.overlay(grid, gadm0_europe_clip_union, how = "intersection")
    # get country attributes
    grid_gadm = gpd.sjoin(grid_intersect, gadm0_europe, how = "left", op = "intersects")
    grid_gadm[BBOX_ID()] = range(len(grid_gadm))
    boxes = grid_gadm.geometry.apply(lambda geom : geom.bounds)
    grid_gadm.geometry = [Polygon(geometry.box(b[0], b[1], b[2], b[3])) for b in boxes] # use the light bboxes
    delete_cols = ["GID_0_left", "NAME_0_left", "a", "continent", "index_right"]
    for column in delete_cols:
        if column in grid_gadm.columns: 
            grid_gadm = grid_gadm.drop(column, 1)
    grid_gadm = grid_gadm.rename(columns = {"GID_0_right":"GID_0", "NAME_0_right":"NAME_0"})
    grid_gadm = grid_gadm[grid_gadm["GID_0"] != "RUS"] # do not include
    grid_gadm.to_file(files["proc_grid"], driver = GEOJSON())
    return grid_gadm

#### Vectors

In [14]:
def transform_lat_lon(lat, lon):
    lat = np.asarray(lat)
    lon = np.asarray(lon)
    trans = Affine.translation(lon[0], lat[0])
    scale = Affine.scale(lon[1] - lon[0], lat[1] - lat[0])
    return trans * scale

def rasterize(polygons, lat, lon, fill=np.nan):
    transform = transform_lat_lon(lat, lon)
    out_shape = (len(lat), len(lon))
    raster = features.rasterize(polygons.geometry, out_shape=out_shape,
                                fill=fill, transform=transform,
                                dtype=float)
    return xr.DataArray(raster, coords={"lat":lat, "lon":lon}, dims=("lat", "lon"))

#### Cubes

#### Arrays

In [15]:
# data np array
# lat_lon dict of "lat" and "lon" holding np arrays of coordinates
def create_xr_dataset(data, lat_lon, name):
    data_array = xr.DataArray(data, coords = list(lat_lon.values()), dims = list(lat_lon.keys()))
    return xr.Dataset({name:data_array})

#### Dates

In [94]:
# date Datetime object to be checked
# weekday String weekday to be checked against
# returns Boolean if date is weekday
def is_weekday(date_x, weekday):
    if not isinstance(date_x, type(datetime.date)): date_x = pd.to_datetime(date_x).date()
    weekday = weekday.lower()[0:3]
    y, m, d = 2000, 1, 3
    wd = ["mon", "tue", "wed", "thu", "fri", "sat", "sun"]
    ref = {}
    for i in range(len(wd)): ref[wd[i]] = datetime(y, m, d + i).date()
    return (date_x - ref[weekday]).days % 7 == 0 # check if date is in a sequence of 7

# Calculates the dates of the periods representing a timestamp each
# n_days_sub Integer how many dates shall one timestamp (sub-period) cover
# baseline Dict start and end dates (datetime) overall baseline period
# target Dict start and end dates (datetime) target period
def calc_periods(n_days_sub, baseline, target):
    fst, lst = "first", "last"
    base_first = baseline[fst]
    n_days = target[fst] - base_first
    n_sub = int(n_days.days / n_days_sub) # number of timestamps
    start = [base_first] * n_sub
    start = [start[i] + timedelta(n_days_sub * i) for i in range(len(start))]
    end = [start[i+1] - timedelta(1) for i in range(len(start)-1)]
    end.append(baseline[lst])
    periods = {"ts":range(len(start)), fst:start, lst:end}
    periods["first"].append(target["first"]) # append start date of target period
    periods["last"].append(target["last"]) # append end date of target period
    return periods

# Returns periods equivalent to target period for other years
# target Dict start and end dates (datetime) target period
# years List of int years
def yearly_period_from_target(target, years = [2018, 2019]):
    fst, lst = "first", "last"
    trgt_fst = target[fst]
    trgt_lst = target[lst]
    start = []
    end = []
    for year in years:
        start.append(datetime(year, trgt_fst.month, trgt_fst.day))
        end.append(datetime(year, trgt_lst.month, trgt_lst.day))
    start.append(trgt_fst)
    end.append(trgt_lst)
    periods = {"ts":range(len(start)), fst:start, lst:end}
    return periods

#### OSM data
Utils for retrieving road data from OSM through its API

In [17]:
# re-order bbox from W,S,E,N to S,W,N,E
def convert_bbox_osm(bbox):
    offset = 0.05 # add a buffer to bbox in order to be sure cube is entirely covered
    bbox_osm = [bbox[1], bbox[0], bbox[3], bbox[2]]
    bbox_osm[0] -= offset # min lon
    bbox_osm[1] += offset # max lat
    bbox_osm[2] += offset # max lon
    bbox_osm[3] -= offset # min lat
    return bbox_osm

# bbox List of four coords
# bbox_id Integer processing id of bbox
# osm_value String OSM value
# osm_key String OSM key
# element_type List of String
# returns GeoPandasDataFrame
def get_osm(bbox, 
            bbox_id,
            osm_value = "motorway",
            osm_key = "highway", # in OSM 'highway' contains several road types: https://wiki.openstreetmap.org/wiki/Key:highway
            element_type = ["way", "relation"]):
    
    bbox_osm = convert_bbox_osm(bbox)
    quot = '"'
    select = quot+osm_key+quot + '=' + quot+osm_value+quot
    select_link = select.replace(osm_value, osm_value + "_link") # also get road links
    select_junction = select.replace(osm_value, osm_value + "_junction")
    geoms = []
    for selector in [select, select_link, select_junction]:  
        try:
            query = overpassQueryBuilder(bbox=bbox_osm, 
                                         elementType=element_type, 
                                         selector=selector, 
                                         out='body',
                                         includeGeometry=True)
            elements = Overpass().query(query, timeout=60).elements()
            # create multiline of all elements
            if len(elements) > 0:
                for i in range(len(elements)):
                    elem = elements[i]
                    geoms.append(elem.geometry())
        except:
            Warning("Could not retrieve " + select)
    try:
        lines = gpd.GeoDataFrame(crs = EPSG_4326(), geometry = geoms)
        n = len(geoms)
        lines[BBOX_ID()] = [bbox_id]*n
        lines["osm_value"] = [osm_value]*n # add road type
        return lines
    except:
        Warning("Could not merge " + osm_value)
        
# buffer Float road buffer distance [m]
# bbox List of four coords
# bbox_id Integer processing id of bbox
# osm_values List of String OSM values
# osm_key String OSM key
# roads_buffer Float buffer width
# dir_write
def get_roads(bbox, bbox_id, osm_values, osm_key, roads_buffer, dir_write):
    fwrite = fname_osm(dir_write, bbox_id, osm_key)
    if not os.path.exists(fwrite):
        roads = []
        has_error = []
        offset = 0.00002
        buffer_dist = "buffer_distance"
        # buffer according to road type
        m,t,p,s,ter = "motorway", "trunk", "primary", "secondary", "tertiary"
        buffers = {m:roads_buffer, t:roads_buffer-offset, p:roads_buffer-(2*offset), s:roads_buffer-(3*offset), ter:roads_buffer-(4*offset)}
        osm_values_int = {m:1, t:2, p:3, s:4, ter:5}
        for osm_value in osm_values:
            try:
                roads_osm = get_osm(bbox = bbox, bbox_id = bbox_id, osm_value = osm_value)
                roads_osm[buffer_dist] = [buffers[osm_value]] * len(roads_osm)
                roads_osm["osm_value_int"] = osm_values_int[osm_value]
                roads.append(roads_osm)
            except:
                has_error.append(1)
                Warning("'get_osm'" + "failed for bbox_id "+ str(bbox_id) + "osm_value " + osm_value + "osm_key" + osm_key)
        if len(roads) > len(has_error):
            roads_merge = gpd.GeoDataFrame(pd.concat(roads, ignore_index=True), crs=roads[0].crs)
            buffered = roads_merge.buffer(distance=roads_merge[buffer_dist])
            roads_merge.geometry = buffered
            roads_merge.to_file(fwrite, driver = GPKG())
    return fwrite

# osm geodataframe of polygons
# reference_raster xarray with lat and lon
def rasterize_osm(osm, reference_raster):
    osm_values = list(set(osm["osm_value"]))
    nan_placeholder = 100
    road_rasters = []
    for osm_value in osm_values:
        osm_subset = osm[osm["osm_value"] == osm_value]
        raster = rasterize(osm_subset, reference_raster.lat, reference_raster.lon)
        cond = np.isfinite(raster)
        raster_osm = np.where(cond, list(osm_subset.osm_value_int)[0], nan_placeholder) # use placeholder instead of nan first
        raster_osm = raster_osm.astype(np.float)
        road_rasters.append(raster_osm)        
    # merge road types in one layer
    road_raster_np = np.array(road_rasters).min(axis=0) # now use the lowest value (highest road level) because some intersect
    road_raster_np[road_raster_np == nan_placeholder] = 0
    return road_raster_np # 0=no_road 1=motorway, 2=trunk, ...

_________
## 5 | Truck detection method

In [80]:
# TruckDetector detects trucks at acquisition-level
class TruckDetector():   
    def __init__(self, band_stack):
        self.band_stack = band_stack
        is_none = band_stack is None
        self.B02 = None if is_none else band_stack.B02 
        self.B03 = None if is_none else band_stack.B03
        self.B04 = None if is_none else band_stack.B04
        self.B08 = None if is_none else band_stack.B08
        self.B11 = None if is_none else band_stack.B11
        self.no_truck_mask = None
        self.trucks = None
        
    def set_trucks(self, trucks):
        self.trucks = trucks # for reload option
    
    # Calculate a binary mask where pixels that are definitely no trucks are represented as 0.
    # thresholds Dict with at least:
    ### max_ndvi Float above this val: no trucks. For Vegetation
    ### max_ndwi Float above this val: no trucks. For Water
    ### max_ndsi Float above this val: no_trucks. For Snow
    ### min_rgb Float above this val: no_trucks. For dark surfaces, e.g. shadows
    ### max_blue Float above this val: no_trucks
    ### max_green Float above this val: no trucks
    ### max_red Float above this val: no trucks
    ### min_b11 Float below this val: no trucks. For dark surfaces, e.g. shadows
    ### max_b11 Float below this val: no trucks. For bright (sealed) surfaces, e.g. buildings
    def calc_no_trucks(self, thresholds):
        B02 = self.B02
        B03 = self.B03
        B04 = self.B04
        B08 = self.B08
        B11 = self.B11
        min_rgb = thresholds["min_rgb"]
        max_blue = thresholds["max_blue"]
        max_green = thresholds["max_green"]
        max_red = thresholds["max_red"]
        max_b11 = thresholds["max_b11"]
        ndvi_mask = ((B08 - B04) / (B08 + B04)) < thresholds["max_ndvi"]
        ndwi_mask = ((B02 - B11) / (B02 + B11)) < thresholds["max_ndwi"]
        ndsi_mask = ((B03 - B11) / (B03 + B11)) < thresholds["max_ndsi"]
        low_rgb_mask = (B02 > thresholds["min_blue"]) * (B03 > min_rgb) * (B04 > min_rgb)
        high_rgb_mask = (B02 < max_blue) * (B03 < max_green) * (B04 < max_red)
        b11_mask = ((B11 - B03) / (B11 + B03)) < max_b11
        b11_mask_abs = (B11 > thresholds["min_b11"]) * (B11 < max_b11)
        self.no_truck_mask = ndvi_mask * ndwi_mask * ndsi_mask * low_rgb_mask * high_rgb_mask * b11_mask * b11_mask_abs
    
    # Calculate a binary mask where trucks are represented as 1 and no trucks as 0.
    # thresholds Dict with at least:
    ### min_green_ratio Float, minimum value of blue-green ratio
    ### min_red_ratio Float, minimum value of blue-red ratio
    def detect_trucks(self, thresholds):
        B02 = self.B02
        B03 = self.B03
        B04 = self.B04
        bg_ratio = (B02 - B03) / (B02 + B03)
        br_ratio = (B02 - B04) / (B02 + B04)
        bg = (bg_ratio * self.no_truck_mask) > thresholds["min_green_ratio"]
        br = (br_ratio * self.no_truck_mask) > thresholds["min_red_ratio"]
        self.trucks = bg * br

_________
## 6 | Processing

In [75]:
# PeriodProcessor processes a period of dates, represented in one cube
class PeriodProcessor():
    def __init__(self, start, end, bbox, bbox_id):
        self.start = start
        self.end = end
        self.bbox = bbox
        self.bbox_id = bbox_id
        self.cube = None
        self.dates = None
        self.lon_lat = None
        self.n_observations = []
        self.detections = []
        self.mean_trucks = None
        self.sum_trucks = None
        self.sum_obs = None
        self.mean_trucks_points = None
        self.dtype = np.uint8
    
    def get_cube(self, dataset, bands, tile_size, spatial_res, day_bins):
        config = CubeConfig(dataset_name = dataset,
                            band_names = bands,
                            tile_size = tile_size,
                            geometry = bbox, # #self.bbox
                            time_range = [start, end], #  # self.start, self.end
                            spatial_res = spatial_res)
        cube = open_cube(config)
        self.cube = cube
        self.dates = cube.time.values
        self.lon_lat = {"lon":cube.lon.values, "lat":cube.lat.values}
        
    # Add from acquisition methods
    def add_n_observations(self, no_clouds):
        obs = np.where(no_clouds == 1, 1, 0)
        self.n_observations.append(obs)
    
    def add_detections(self, trucks):
        self.detections.append(trucks)
    
    # Temporal summary methods
    def sum_truck_n(self):
        self.sum_trucks = np.array(self.detections).sum(axis=0)
    
    def sum_observations(self):
        self.sum_observations = np.array(self.n_observations).sum(axis=0)
            
    def mean_truck_n(self): 
        self.mean_trucks = np.round(self.sum_trucks / self.n_obs) # first round
                
    def vectorize_mean_trucks(self, crs):
        match_value = np.array(1, dtype=self.mean_trucks.dtype)
        self.mean_trucks_points = points_from_np(self.mean_trucks, match_value, self.lon_lat, crs=crs)
    
    # Write methods
    def write_n_observations(self, dirs_ts, bbox_id, ext):
        fname = fname_sum_obs(dirs_ts, bbox_id, ext)
        sum_obs_xr = create_xr_dataset(self.sum_obs, self.lon_lat, os.path.basename(fname))
        sum_obs_xr.to_netcdf(sum_obs_xr, fname)
        
    def write_sum_trucks(self, dirs_ts, bbox_id, ext):
        fname = fname_sum_trucks(dirs_ts, bbox_id, ext)
        sum_xr = create_xr_dataset(self.sum_trucks, self.lon_lat, os.path.basename(fname))
        sum_xr.to_netcdf(sum_xr, fname)
    
    def write_mean_trucks(self, dirs_ts, bbox_id, ext):
        fname = fname_mean_trucks(dirs_ts, bbox_id, ext)
        mean_xr = create_xr_dataset(self.mean_trucks, self.lon_lat, os.path.basename(fname))
        mean_xr.to_netcdf(mean_xr, fname)
        
    def write_mean_trucks_vec(self, dirs_ts, bbox_id, ext):
        n = self.mean_trucks_points is None or len(self.mean_trucks_points)
        if n > 0:
            fname = fname_mean_trucks_vec(dirs_ts, bbox_id, ext)
            driver = GPKG() if ext == GPKG_EXT() else None
            driver = GEOJSON() if ext == GEOJSON_EXT() else None
            if driver is None: driver = GPKG()
            self.mean_trucks_points.to_file(fname, driver = driver)
        else:
            # write txt as placeholder
            fname = fname_mean_trucks_vec_placeholder(dirs_ts, bbox_id, ".txt")
            with open(fname) as file:
                file.write("'mean_trucks_points' has length: %s. Nothing to write" %(str(n)))
            
    def wrap_period(self, dirs_ts, bbox_id, ext_arr = NC_EXT(), ext_vec = GPKG_EXT()):
        self.sum_observations()
        self.sum_truck_n()
        self.mean_truck_n()
        try:
            self.vectorize_mean_trucks(EPSG_4326())
        except:
            Warning("points_from_np failed")
        self.write_n_observations(dirs_ts, bbox_id, ext_arr)
        self.write_sum_trucks(dirs_ts, bbox_id, ext_arr)
        self.write_mean_trucks(dirs_ts, bbox_id, ext_arr)
        self.write_mean_trucks_vec(dir_ts, bbox_id, ext_vec)

In [76]:
# AcquisitionProcessor processes all valid pixels of a single acquisition in cube
class AcquisitionProcessor():
    def __init__(self, date_np64, cube):
        self.date_np64 = date_np64
        self.cube = cube
        self.band_stack = cube.sel(time = date_np64)
        self.detector = None
        self.no_clouds = None
        
    def mask_clouds(self):
        scl = MaskSet(self.band_stack.SCL)
        high_prob = scl.clouds_high_probability
        med_prob = scl.clouds_medium_probability
        low_prob = scl.clouds_low_probability_or_unclassified
        cirrus = scl.cirrus
        shadows = scl.cloud_shadows
        dark = scl.dark_area_pixels
        self.no_clouds = (high_prob + med_prob + low_prob + cirrus + shadows + dark) == 0
        self.band_stack = self.band_stack.where(self.no_clouds)
    
    # percentage Float secifies the percentage of valid observations
    # that is needed to be considered as valid acquisition
    def has_observations(self, minimum_valid_percentage):
        values = self.no_clouds.values.flatten()
        n_vals = len(values)
        n_valid = np.count_nonzero(values)
        percent_valid = (n_valid / n_vals) * 100
        return percent_valid >= minimum_valid_percentage
    
    # osm gpd polygons
    def mask_with_osm(self, osm):
        osm_raster = rasterize_osm(osm, self.band_stack.B02)
        mask_xr = create_xr_dataset(osm_raster, {"lat":self.cube.lat.values, "lon":self.cube.lon.values}, "roadmask")
        self.band_stack = self.band_stack.where(mask_xr.roadmask != 0)
                
    def do_detection(self, thresholds):
        self.detector = TruckDetector(self.band_stack)
        self.detector.calc_no_trucks(thresholds)
        self.detector.detect_trucks(thresholds)
        
    def write_detections(self, fname):
        trucks_xr = create_xr_dataset(self.detector.trucks, {"lat":self.cube.lat.values, "lon":self.cube.lon.values},
                                    os.path.basename(fname))
        # add no_clouds layer (n observations) for possible reload
        trucks_xr_with_no_clouds = trucks_xr.assign({"no_clouds":self.no_clouds})
        trucks_xr_with_no_clouds.to_netcdf(fname)
        
    def read_detections(self, fname):
        dataset = xr.open_dataset(fname)
        self.no_clouds = dataset["no_clouds"]
        trucks = dataset[os.path.basename(fname)]
        self.detector = TruckDetector(None)
        self.detector.set_trucks(trucks)

__________
____________
## 7 | Do Processing
Process data by __weekday__, __timestamp__ (sub period) processing grid __box__ (bbox_id).

In [23]:
# make or read processing grid
if os.path.exists(files["proc_grid"]):
    try:
        grid_gadm = gpd.read_file(files["proc_grid"])
    except:
        raise Exception("Failed reading proc grid from: " + files["proc_grid"])
else:
    grid_gadm = make_grid(grid_spacing, files)
# calc temporal bounds of baseline sub-periods
if len(baseline_years) > 0:
    periods = yearly_period_from_target(target, baseline_years)
else:
    periods = calc_periods(n_days_sub, baseline, target)

#### Main process

In [ ]:
trace = []
ext_arr = NC_EXT()
ext_vec = GPKG_EXT()
sep = "-" * 20
for i in range(len(grid_gadm)):
    i = list(grid_gadm[BBOX_ID()]).index(745)
    bbox = grid_gadm.geometry[i].bounds    
    bbox_id = grid_gadm[BBOX_ID()][i]
    bbox_id_str = str(bbox_id)
    first = periods["first"]
    last = periods["last"]
    print("%s\nProcessing: %s\nbbox_id: %s" %(sep, str(i), bbox_id_str))
    try:
        file_osm = get_roads(bbox, bbox_id, osm_values, osm_key, roads_buffer, dirs["ancil_roads"])
        # retry
        if not os.path.exists(file_osm): file_osm = get_roads(bbox, bbox_id, osm_values, osm_key, roads_buffer, dirs["ancil_roads"])
        osm = gpd.read_file(file_osm)
    except:
        msg = "Could not get OSM roads: " + bbox_id_str
        Warning(msg)
        trace.append(msg)
        continue
    for start, end in zip(first, last):
        
        test_t1 = datetime.now()
        
        ts = first.index(start)
        ts_str = str(ts)
        dir_ts = os.path.join(dirs["processed"], "ts_%s_%s_%s" %(ts_str, str(start.date()), str(end.date())))
        if not os.path.exists(dir_ts): os.mkdir(dir_ts)
        dirs_ts = make_dirs_ts(dir_ts)
        print("TS: %s Period Start: %s End: %s" %(ts_str, str(start), str(end)))
        
        # check if yet processed
        fname_sum_observations = fname_sum_obs(dirs_ts, bbox_id, ext_arr)
        fname_sum = fname_sum_trucks(dirs_ts, bbox_id, ext_arr)
        fname_mean = fname_mean_trucks(dirs_ts, bbox_id, ext_arr)
        fname_mean_vec = fname_mean_trucks_vec(dirs_ts, bbox_id, ext_vec)
        fname_mean_vec_placeholder = fname_mean_trucks_vec_placeholder(dirs_ts, bbox_id)
        exist = [os.path.exists(file) for file in [fname_sum_observations, fname_sum, fname_mean, fname_mean_vec]]
        if not exist[3]: exist[3] = os.path.exists(fname_mean_vec_placeholder) # placeholder might exist instead
        
        already_proc = "because already processed"
        if all(exist) and not overwrite_results:
            print("Skipping " + already_proc)
        else:
            period = PeriodProcessor(start, end, bbox, bbox_id)
            period.get_cube(dataset, bands, tile_size, spatial_res, day_bins)
            t1 = datetime.now() # track time cube is open to prevent timeout
            n_acquisitions = 0
            for period_date in period.dates:
                acquisition = AcquisitionProcessor(period_date, period.cube)
                date_str = str(period_date)
                fname = fname_acquisition_trucks(dirs_ts, bbox_id, date_str, ext_arr)
                if os.path.exists(fname):
                    try:
                        acquisition.read_detections(fname)
                        period.add_n_observations(acquisition.no_clouds)
                        period.add_detections(acquisition.detector.trucks.values)
                        n_acquisitions += 1
                        print("Read processed date: %s %s" %(date_str, already_proc))
                    except:
                        Warning("Could not read: " + fname)
                else:
                    weekday_match = any([is_weekday(period_date, w) for w in weekdays])
                    if weekday_match:
                        acquisition.mask_clouds()
                        # check if enough observations to be considered
                        if acquisition.has_observations(minimum_valid_observations): 
                            n_acquisitions += 1
                            acquisition.mask_with_osm(osm) # mask to OSM roads
                            acquisition.do_detection(thresholds) # truck detection
                            print("Writing")
                            try:
                                acquisition.write_detections(fname)
                            except:
                                Warning("Failed writing: " + fname)
                            period.add_n_observations(acquisition.no_clouds)
                            period.add_detections(acquisition.detector.trucks.values)
                            if not os.path.exists(fname): print("File was not written: " + fname)
                            print("Done with date: %s" %(date_str))
                        else:
                            continue
                    else:
                        continue
                t2 = datetime.now()
                if (t2-t1).seconds > 3600: # if one hour is exceeded, be sure no timeout
                    period.get_cube(dataset, bands, tile_size, spatial_res, day_bins)
            duration = datetime.now() - test_t1
            if n_acquisitions > 0:
                period.wrap_period(dirs_ts, bbox_id, arr_ext, ext_vec)
            else:
                msg = "No acquisitions in period %s to %s. In bbox_id: %s" %(str(start), str(end), bbox_id_str)
                Warning(msg)
                trace.append(msg)
        print("Done with period %s of bbox_id %s\n%s" %(ts_str, bbox_id_str, sep))
    print("Done with bbox_id: " + bbox_id_str)
    